In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# #weights = [0,0,-1,-1,
#            0,-1,-1,0,
#            0,0,
#            1,-1,
#            1,-1,
#            -1,-1,
#            1,1]
#set weights list to all zeroes
# weights = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#set weights to a list of 18 1s
weights = [-1,1,0,0,
            0, 1, -1, 0,
            -1, -1, 1, -1,
            -1, 1, 0, 0,
            1, 1]

biases = [0.23568961381660444, -0.2103954962539835, 0.20839107539344723, 0.0, 
          -0.43799690348173953, 0.3913951196424467, -0.1418436937100001]
#biases = [0,0,0,0,0,0,0]

x_input_range = [0, .9]
y_input_range = [0, 1]

#Element-wise ReLU
def relu(x):
    return np.maximum(np.zeros_like(x), x)


def crn(x1p, x1m, x2p, x2m,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1
):
    w = np.array([[w11, w12, w13, w14],
                    [w21, w22, w23, w24]], dtype='float64')
    u = np.array([[u11, u12],
                    [u21, u22],
                    [u31, u32],
                    [u41, u42]], dtype='float64')
    v = np.array([v11,
                    v21], dtype='float64')
    a = np.array([a1, a2, a3, a4], dtype='float64')
    b = np.array([b1, b2], dtype='float64'),
    c = np.array([c1], dtype='float64')
    X = np.array([[x1p,x1m], [x2p,x2m]], dtype='float64')
    A = np.array([[relu(a1),relu(-a1)],
                 [relu(a2),relu(-a2)],
                 [relu(a3),relu(-a3)],
                 [relu(a4),relu(-a4)]], dtype='float64')
    B = np.array([[0,0],
                 [0,0],
                 [0,0],
                 [0,0]], dtype='float64')
    C = np.array([[relu(b1),relu(-b1)],
                 [relu(b2),relu(-b2)]], dtype='float64')
    D = np.array([[0,0],
                 [0,0]], dtype='float64')
    Z = np.array([relu(c1),relu(-c1)], dtype='float64')
    #First layer
    for i in range(2):
        for j in range(4):
            if(w[i][j] == 1):
                A[j][0] = np.add(A[j][0], X[i][0], dtype='float64')
                A[j][1] = np.add(A[j][1], X[i][1], dtype='float64')
            elif(w[i][j] == -1):
                A[j][0] = np.add(A[j][0], X[i][1], dtype='float64')
                A[j][1] = np.add(A[j][1], X[i][0], dtype='float64')
    #First ReLU layer
    for i in range(4):
        B[i][0] = A[i][0]
        B[i][1] = np.minimum(A[i][0], A[i][1], dtype='float64')
    for i in range(4):
        for j in range(2):
            if(u[i][j] == 1):
                C[j][0] = np.add(C[j][0], B[i][0], dtype='float64')
                C[j][1] = np.add(C[j][1], B[i][1], dtype='float64')
            elif(u[i][j] == -1):
                C[j][0] = np.add(C[j][0], B[i][1], dtype='float64')
                C[j][1] = np.add(C[j][1], B[i][0], dtype='float64')
    for i in range(2):
        D[i][0] = C[i][0]
        D[i][1] = np.minimum(C[i][0], C[i][1], dtype='float64')
    for i in range(2):
        if(v[i] == 1):
            Z[0] = np.add(Z[0], D[i][0], dtype='float64')
            Z[1] = np.add(Z[1], D[i][1], dtype='float64')
        elif(v[i] == -1):
            Z[0] = np.add(Z[0], D[i][1], dtype='float64')
            Z[1] = np.add(Z[1], D[i][0], dtype='float64')
    return Z


def crnp(x1p, x1m, x2p, x2m,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1
):
    return crn(x1p, x1m, x2p, x2m,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1)[0]
    

def crnm(x1p, x1m, x2p, x2m,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1
):
    return crn(x1p, x1m, x2p, x2m,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1)[1]


def ntwk(x1, x2,
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1
):
    w = np.array([[w11, w12, w13, w14],
                    [w21, w22, w23, w24]])
    u = np.array([[u11, u12],
                    [u21, u22],
                    [u31, u32],
                    [u41, u42]])
    v = np.array([[v11],
                    [v21]])
    a = np.array([a1, a2, a3, a4])
    b = np.array([b1, b2]),
    c = np.array([c1])
    layer1out = relu((np.array([x1, x2]) @ w) + a)
    layer2out = relu((layer1out @ u) + b)
    layer3out = (layer2out @ v) + c
    return np.ndarray.item(layer3out)

vntwk = np.vectorize(ntwk)
vcrnp = np.vectorize(crnp)
vcrnm = np.vectorize(crnm)


def plotntwk(
        #weights
        w11, w12, w13, w14, w21, w22, w23, w24,
        u11, u12, u21, u22, u31, u32, u41, u42,
        v11, v21,
        #biases
        a1, a2, a3, a4,
        b1, b2,
        c1,
        cutoff):
    x = np.concatenate((np.flip(np.linspace(x_input_range[0], x_input_range[1], 6)), np.linspace(x_input_range[0], x_input_range[1], 6)))
    y = np.linspace(y_input_range[0], y_input_range[1], 8)
    X, Y = np.meshgrid(x, y)
    xp = np.maximum(np.zeros_like(X), X)
    xm = np.maximum(np.zeros_like(X), np.negative(X))
    yp = np.maximum(np.zeros_like(Y), Y)
    ym = np.maximum(np.zeros_like(Y), np.negative(Y))
    xouts = np.stack((xp, xm), axis=2)
    youts = np.stack((yp, ym), axis=2)
    Xp, Yp = np.meshgrid(xp, yp)
    Xm, Ym = np.meshgrid(xm, ym)
    data = vntwk(X, Y,
                w11, w12, w13, w14, w21, w22, w23, w24,
                u11, u12, u21, u22, u31, u32, u41, u42,
                v11, v21,
                #biases
                a1, a2, a3, a4,
                b1, b2,
                c1)
    binarydata = np.where(data < cutoff, 0, 1)

    pdata = vcrnp(xp, xm, yp, ym,
                w11, w12, w13, w14, w21, w22, w23, w24,
                u11, u12, u21, u22, u31, u32, u41, u42,
                v11, v21,
                #biases
                a1, a2, a3, a4,
                b1, b2,
                c1)

    mdata = vcrnm(xp, xm, yp, ym,
                w11, w12, w13, w14, w21, w22, w23, w24,
                u11, u12, u21, u22, u31, u32, u41, u42,
                v11, v21,
                #biases
                a1, a2, a3, a4,
                b1, b2,
                c1)

    #UT color map:
    N = 256
    vals = np.ones((N,4))
    vals[:,0] = np.linspace(1,191/256,N)
    vals[:,1] = np.linspace(1,87/256,N)
    vals[:,2] = np.linspace(1,0/256,N)
    cmap = matplotlib.colors.ListedColormap(vals)

    print(f'x-axis input range: {x_input_range}')
    print(f'y-axis input range: {y_input_range}')
    print(f'Largest Z+ amount: {np.amax(pdata)}')
    print(f'Largest Z- amount: {np.amax(mdata)}')
    print(f'Dual-rail value closest to 0 (smallest difference between Z+ and Z-): {np.amin(np.abs(data))}')
    print(f'Range of dual-rail outputs: {np.amin(data)} to {np.amax(data)}')
    print(f'Range of positive outputs: {np.amin(pdata)} to {np.amax(pdata)}')
    print(f'Range of negative outputs: {np.amin(mdata)} to {np.amax(mdata)}')
    
    plt.subplots(figsize=(24,16))
    ax = plt.subplot(1,4,1)
    img = plt.imshow(data, cmap=cmap.reversed())
    for (j,i),label in np.ndenumerate(data):
        ax.text(i,j,np.around(label,1),ha='center',va='center')
    #plt.colorbar(img, fraction=0.03)
    plt.xticks([], [])
    plt.yticks([], [])

    plt.subplot(1,4,2)
    plt.imshow(binarydata, cmap=cmap.reversed())
    plt.xticks([], [])
    plt.yticks([], [])
    

    bx = plt.subplot(1,4,3)
    for (j,i),label in np.ndenumerate(pdata):
        bx.text(i,j,np.around(label,1),ha='center',va='center')
    plt.imshow(pdata, cmap=cmap.reversed())
    plt.xticks([], [])
    plt.yticks([], [])
    

    cx = plt.subplot(1,4,4)
    for (j,i),label in np.ndenumerate(mdata):
        cx.text(i,j,np.around(label,1),ha='center',va='center')
    plt.imshow(mdata, cmap=cmap.reversed())
    plt.xticks([], [])
    plt.yticks([], [])
    plt.show()
    plt.close()
    

# inputSliders = []
# for i in range(2):
#     inputSliders.append(widgets.FloatSlider(min=-1, max=1, step=.01))
weightSliders = []
for w in weights:
    weightSliders.append(widgets.IntSlider(min=-1, max=1, step=1, value=w, layout=widgets.Layout(width='20%')))
biasSliders = []
for b in biases:
    biasSliders.append(widgets.FloatSlider(min=-7, max=7, step=.01, value=b))

widget = widgets.interactive(plotntwk, 
        #x1=inputSliders[0], x2=inputSliders[1],
        #weights
        w11=weightSliders[0], w12=weightSliders[1], w13=weightSliders[2], w14=weightSliders[3], w21=weightSliders[4], w22=weightSliders[5], w23=weightSliders[6], w24=weightSliders[7],
        u11=weightSliders[8], u12=weightSliders[9], u21=weightSliders[10], u22=weightSliders[11], u31=weightSliders[12], u32=weightSliders[13], u41=weightSliders[14], u42=weightSliders[15],
        v11=weightSliders[16], v21=weightSliders[17],
        #biases
        a1=biasSliders[0], a2=biasSliders[1], a3=biasSliders[2], a4=biasSliders[3],
        b1=biasSliders[4], b2=biasSliders[5],
        c1=biasSliders[6],
        cutoff=widgets.FloatSlider(min=-10, max=10, step=.1))
layer1weights = widgets.HBox(widget.children[0:8], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
layer2weights = widgets.HBox(widget.children[8:16], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
layer3weights = widgets.HBox(widget.children[16:18], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
layer1bias = widgets.HBox(widget.children[18:22], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
layer2bias = widgets.HBox(widget.children[22:24], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
layer3biasandcutoff = widgets.HBox(widget.children[24:26], layout = widgets.Layout(flex_flow='row wrap', border='solid 1px'))
output = widget.children[-1]
widgets.VBox([layer1weights, layer2weights, layer3weights, layer1bias, layer2bias, layer3biasandcutoff, output])